In [4]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers import Input, Lambda
from keras import losses
from keras.models import model_from_json
import keras.backend.tensorflow_backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [5]:
#dir = './sequence/*'
dir = './latest_sequence/bfs-character/*'

In [6]:
# file read
name = []
all_data = []
sequence_length = []
alpha = list(string.ascii_uppercase)
for file in sorted(glob.glob(dir)):
    name.append(file.split('/')[-1].replace('.txt', ''))
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(', ')
        datasets.append([alpha.index(u[1])+1, alpha.index(v[1]) +1, float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [7]:
x_train, x_test, train_name, test_name = train_test_split(all_data, name, test_size=0.3)
x_test, x_val, test_name, val_name = train_test_split(x_test, test_name, test_size=0.33)

In [8]:
train_name
tr_names= []
for name in train_name:
    tr_names.append(name.split('-')[0].replace('graph', ''))

In [9]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 500
steps_per_epoch = len(x_train)

In [10]:
def custom_loss(y_true, y_pred):
    loss1 = losses.mean_squared_error(y_true, y_pred)
    #loss2 = losses.kld(y_true, y_pred)
    return loss1# * 0.7 + loss2 * 0.3

In [8]:
def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, 3))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
encoded = LSTM(64)(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(3))(decoded)
encoder = Model(inputs, encoded)

lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(loss=losses.mean_squared_error, optimizer='adam')
#lstm_autoencoder_500 = lstm_autoencoder

W1007 23:51:24.764862 139985173763840 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1007 23:51:24.788869 139985173763840 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1007 23:51:24.794222 139985173763840 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1007 23:51:25.861754 139985173763840 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [9]:
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0

In [11]:
model_path = 'model_save/mse_weights/weights' + '{epoch:02d}-{loss:.4f}.h5'
early_stopping_callback = EarlyStopping(monitor='loss', patience=200)
checkpoint_callback = ModelCheckpoint(model_path, monitor='loss', verbose = 1, save_best_only = True, save_weights_only = True, mode='min')#, period=5)

In [12]:
lstm_autoencoder.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=1, callbacks=[early_stopping_callback, checkpoint_callback])

Epoch 1/500
2360/2360 [==============================] - 76s 32ms/step - loss: 37.1500

Epoch 00001: loss improved from inf to 37.14996, saving model to result/mse_weights/weights01-37.1500.h5
Epoch 2/500
2360/2360 [==============================] - 63s 27ms/step - loss: 37.1118

Epoch 00002: loss improved from 37.14996 to 37.11185, saving model to result/mse_weights/weights02-37.1118.h5
Epoch 3/500
2360/2360 [==============================] - 46s 19ms/step - loss: 37.0955

Epoch 00003: loss improved from 37.11185 to 37.09550, saving model to result/mse_weights/weights03-37.0955.h5
Epoch 4/500
2360/2360 [==============================] - 39s 16ms/step - loss: 37.0872

Epoch 00004: loss improved from 37.09550 to 37.08720, saving model to result/mse_weights/weights04-37.0872.h5
Epoch 5/500
2360/2360 [==============================] - 48s 20ms/step - loss: 37.0588

Epoch 00005: loss improved from 37.08720 to 37.05875, saving model to result/mse_weights/weights05-37.0588.h5
Epoch 6/500
236

2360/2360 [==============================] - 35s 15ms/step - loss: 0.7549

Epoch 00100: loss did not improve from 0.63263
Epoch 101/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.7406

Epoch 00101: loss did not improve from 0.63263
Epoch 102/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.6796

Epoch 00102: loss did not improve from 0.63263
Epoch 103/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.6602

Epoch 00103: loss did not improve from 0.63263
Epoch 104/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.6246

Epoch 00104: loss improved from 0.63263 to 0.62464, saving model to result/mse_weights/weights104-0.6246.h5
Epoch 105/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.5552

Epoch 00105: loss improved from 0.62464 to 0.55523, saving model to result/mse_weights/weights105-0.5552.h5
Epoch 106/500
2360/2360 [==============================] - 35s 15ms/step - l

2360/2360 [==============================] - 35s 15ms/step - loss: 0.1429

Epoch 00211: loss did not improve from 0.13178
Epoch 212/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.1592

Epoch 00212: loss did not improve from 0.13178
Epoch 213/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1698

Epoch 00213: loss did not improve from 0.13178
Epoch 214/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.1280

Epoch 00214: loss improved from 0.13178 to 0.12796, saving model to result/mse_weights/weights214-0.1280.h5
Epoch 215/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1544

Epoch 00215: loss did not improve from 0.12796
Epoch 216/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.1445

Epoch 00216: loss did not improve from 0.12796
Epoch 217/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1339

Epoch 00217: loss did not improve from 0.12796
E

2360/2360 [==============================] - 35s 15ms/step - loss: 0.1070

Epoch 00269: loss did not improve from 0.10376
Epoch 270/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1126

Epoch 00270: loss did not improve from 0.10376
Epoch 271/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.1282

Epoch 00271: loss did not improve from 0.10376
Epoch 272/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1054

Epoch 00272: loss did not improve from 0.10376
Epoch 273/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.1146

Epoch 00273: loss did not improve from 0.10376
Epoch 274/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.1009

Epoch 00274: loss improved from 0.10376 to 0.10089, saving model to result/mse_weights/weights274-0.1009.h5
Epoch 275/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.1052

Epoch 00275: loss did not improve from 0.10089
E

2360/2360 [==============================] - 36s 15ms/step - loss: 0.0908

Epoch 00327: loss did not improve from 0.08278
Epoch 328/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0970

Epoch 00328: loss did not improve from 0.08278
Epoch 329/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0900

Epoch 00329: loss did not improve from 0.08278
Epoch 330/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0845

Epoch 00330: loss did not improve from 0.08278
Epoch 331/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.1038

Epoch 00331: loss did not improve from 0.08278
Epoch 332/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0827

Epoch 00332: loss improved from 0.08278 to 0.08268, saving model to result/mse_weights/weights332-0.0827.h5
Epoch 333/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0863

Epoch 00333: loss did not improve from 0.08268
E

2360/2360 [==============================] - 36s 15ms/step - loss: 0.1705

Epoch 00385: loss did not improve from 0.06702
Epoch 386/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.1797

Epoch 00386: loss did not improve from 0.06702
Epoch 387/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1146

Epoch 00387: loss did not improve from 0.06702
Epoch 388/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0696

Epoch 00388: loss did not improve from 0.06702
Epoch 389/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0651

Epoch 00389: loss improved from 0.06702 to 0.06509, saving model to result/mse_weights/weights389-0.0651.h5
Epoch 390/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0718

Epoch 00390: loss did not improve from 0.06509
Epoch 391/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0833

Epoch 00391: loss did not improve from 0.06509
E

2360/2360 [==============================] - 36s 15ms/step - loss: 0.0691

Epoch 00444: loss did not improve from 0.05706
Epoch 445/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.0672

Epoch 00445: loss did not improve from 0.05706
Epoch 446/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0666

Epoch 00446: loss did not improve from 0.05706
Epoch 447/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0696

Epoch 00447: loss did not improve from 0.05706
Epoch 448/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.1379

Epoch 00448: loss did not improve from 0.05706
Epoch 449/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.1175

Epoch 00449: loss did not improve from 0.05706
Epoch 450/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0613

Epoch 00450: loss did not improve from 0.05706
Epoch 451/500
2360/2360 [==============================] - 35s

In [13]:
model_json = lstm_autoencoder.to_json()
filename = 'last_mse_lstmae' # input('filename: ') #
with open('model_save/mse_weights/' + filename + '.json', 'w') as file:
    file.write(model_json)
lstm_autoencoder.save_weights('model_save/mse_weights/weights_' +  filename + '.h5')

In [12]:
filename = "last_mse_lstmae"
loaded_model = model_from_json(open('model_save/mse_weights/' +filename + '.json').read())
loaded_model.load_weights('model_save/mse_weights/weights_' + filename + '.h5')

W1010 15:48:40.110543 140189195294464 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1010 15:48:40.120297 140189195294464 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1010 15:48:40.123097 140189195294464 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1010 15:48:40.688915 140189195294464 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [13]:
loaded_model = lstm_autoencoder

NameError: name 'lstm_autoencoder' is not defined

In [14]:
mean= 0
for xt in x_test:
    xt = xt.reshape(1, xt.shape[0], xt.shape[1])
    out = loaded_model.predict(xt)
    mean += ((xt-out)**2).mean(axis=None)
print(mean/len(x_test))

0.7717624475759962


In [ ]:
encoder = Model(loaded_model.input, loaded_model.layers[3].output)

In [ ]:
test = x_test[0].reshape(1, x_test[0].shape[0], x_test[0].shape[1])
latent_vector = []
for x in x_test:
    x = x.reshape(1, x.shape[0], x.shape[1])
    latent_vector.append(encoder.predict(x)[0])